# Co Simulation

In [1]:
import numpy as np
import pandas as pd

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=30)

In [4]:
curve = pd.read_csv('./../cases/Curve.csv')

In [ ]:
sp = ams.load('./../cases/IL200_rted.xlsx',
              setup=True, no_output=True,
              default_config=True)

sa = sp.to_andes(addfile='./../cases/IL200_dyn_db.xlsx',
                 setup=False,
                 no_output=True,
                 default_config=True,)

Generating code for 1 models on 12 processes.


Following PFlow models in addfile will be overwritten: <Bus>, <PQ>, <PV>, <Slack>, <Shunt>, <Line>, <Area>
AMS system 0x136a81f40 is linked to the ANDES system 0x33c2ff9e0.


In [ ]:
# set Wind and Solar to be uncontrollable
stg_wind, stg_pv = sp.StaticGen.find_idx(keys='genfuel',
                                         values=['wind', 'solar'], allow_all=True)
sp.StaticGen.set(src='ctrl', attr='v', idx=stg_wind, value=0)
sp.StaticGen.set(src='ctrl', attr='v', idx=stg_pv, value=0)

# set load levels
p0 = sp.PQ.get(src='p0', attr='v', idx=sp.PQ.idx.v).copy()
sp.PQ.set(src='p0', attr='v', idx=sp.PQ.idx.v,
          value=curve['Load'].values[0:5].mean() * p0,
          #    value=0.85 * p0,
          )

# set wind power
p0_wind = sp.StaticGen.get(src='p0', attr='v', idx=stg_wind).copy()
sp.StaticGen.set(src='p0', attr='v', idx=stg_wind,
                 value=curve['Wind'].values[0:5].mean() * p0_wind)

# set solar power
p0_pv = sp.StaticGen.get(src='p0', attr='v', idx=stg_pv).copy()
sp.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                 value=curve['PV'].values[0:5].mean() * p0_pv)

stg = sp.StaticGen.get_all_idxes()
# pg0 <- p0, relax RTED ramping constraints
sp.StaticGen.set(src='pg0', attr='v', idx=stg,
                 value=sp.StaticGen.get(src='p0', attr='v', idx=stg))
# relax StaticGen.pmin
sp.StaticGen.set(src='pmin', attr='v', idx=stg, value=0)

sp.RTED.run(solver='CLARABEL')

In [ ]:
sp.RTED.dc2ac()

In [ ]:
sp.dyn.send(adsys=sa, routine='RTED')

In [ ]:
# Constant load
sa.PQ.config.p2p = 1
sa.PQ.config.q2q = 1
sa.PQ.config.p2z = 0
sa.PQ.config.q2z = 0

sa.PFlow.run()

In [ ]:
sa.TDS.config.criteria = 0
sa.TDS.config.no_tqdm = 1
_ = sa.TDS.init()

In [ ]:
sa.TDS.run()

In [ ]:
sa.ACEc.bias.pu_coeff

In [ ]:
omega = sa.GENROU.omega.v[sa.GENROU.idx2uid('GENROU_47')]
bias = - 0.3 * sa.config.mva / (10 * sa.config.freq * (omega - 1))
bias

In [ ]:

fmin, fmax = 59.4, 60.01
tmin, tmax = 0, 8
linewidth = 1.5
genrou_idx = 'GENROU_47'

# left: short-term response, without and with thermal inertia
_ = sa.TDS.plt.plot(
    sa.GENROU.omega,
    a=sa.GENROU.idx2uid(genrou_idx),
    left=tmin, right=tmax,
    ymin=fmin, ymax=fmax,
    ytimes=sa.config.freq,
    show=False, grid=True,
    ylabel='Slack Gen. Freq. [Hz]',
)